In [1]:
import numpy as np
import utils
import network_layers
import tensorflow as tf
from keras import layers, Model

In [3]:
#set the necessary seeds
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

## Import datasets

### Breast cancer dataset

In [4]:
train_pairs_breast, val_pairs_breast, test_pairs_breast = utils.split_dataset(utils.create_list())
print(f"Number of training pairs: {len(train_pairs_breast)}")
print(f"Number of val pairs: {len(val_pairs_breast)}")
print(f"Number of test_pairs pairs: {len(test_pairs_breast)}")

Number of training pairs: 388
Number of val pairs: 129
Number of test_pairs pairs: 130


In [5]:
train_dataset_breast = utils.create_dataset(train_pairs_breast, augment=False)
val_dataset_breast = utils.create_dataset(val_pairs_breast)
test_dataset_breast = utils.create_dataset(test_pairs_breast)

In [6]:
batch_size = 8

In [7]:
train_dataset_breast = train_dataset_breast.shuffle(buffer_size=len(train_pairs_breast)).batch(batch_size)
val_dataset_breast = val_dataset_breast.batch(batch_size)
test_dataset_breast = test_dataset_breast.batch(batch_size)

### Skin cancer dataset

In [4]:
train_dataset_skin = utils.create_dataset(utils.create_list_skin("Datasets/ISIC-2017_Training_Data", "Datasets/ISIC-2017_Training_Part1_GroundTruth"), augment=True)
val_dataset_skin = utils.create_dataset(utils.create_list_skin("Datasets/ISIC-2017_Validation_Data", "Datasets/ISIC-2017_Validation_Part1_GroundTruth"))
test_dataset_skin =utils. create_dataset(utils.create_list_skin("Datasets/ISIC-2017_Test_v2_Data", "Datasets/ISIC-2017_Test_v2_Part1_GroundTruth"))

In [7]:
train_dataset_skin = train_dataset_skin.shuffle(buffer_size=2000).batch(batch_size)
val_dataset_skin = val_dataset_skin.batch(batch_size)
test_dataset_skin = test_dataset_skin.batch(batch_size)

### Brain cancer dataset

In [4]:
train_pairs_brain, val_pairs_brain, test_pairs_brain = utils.split_dataset(utils.create_list_brain())
print(f"Number of training pairs: {len(train_pairs_brain)}")
print(f"Number of val pairs: {len(val_pairs_brain)}")
print(f"Number of test_pairs pairs: {len(test_pairs_brain)}")

Number of training pairs: 1838
Number of val pairs: 613
Number of test_pairs pairs: 613


In [5]:
train_dataset_brain = utils.create_dataset(train_pairs_brain, augment=True)
val_dataset_brain = utils.create_dataset(val_pairs_brain)
test_dataset_brain = utils.create_dataset(test_pairs_brain)

In [7]:
train_dataset_brain = train_dataset_brain.shuffle(buffer_size=len(train_pairs_brain)).batch(batch_size)
val_dataset_brain = val_dataset_brain.batch(batch_size)
test_dataset_brain = test_dataset_brain.batch(batch_size)

## Architecture definition

In [8]:
def UCTransNet(input_shape=(256, 256, 3), num_filters=16):
    inputs = layers.Input(shape=input_shape)

    #encoder (as in the original U-Net)
    first_conv = network_layers.convolutional_layer(inputs, num_filters)
    first_pool = layers.MaxPooling2D(pool_size=(2, 2))(first_conv)

    second_conv = network_layers.convolutional_layer(first_pool, num_filters * 2)
    second_pool = layers.MaxPooling2D(pool_size=(2, 2))(second_conv)

    third_conv = network_layers.convolutional_layer(second_pool, num_filters * 4)
    third_pool = layers.MaxPooling2D(pool_size=(2, 2))(third_conv)

    fourth_conv = network_layers.convolutional_layer(third_pool, num_filters * 8)
    fourth_pool = layers.MaxPooling2D(pool_size=(2, 2))(fourth_conv)

    #bottleneck
    fifth_conv = network_layers.convolutional_layer(fourth_pool, num_filters * 16)

    #apply ChannelTransformer on the bottleneck
    transformer_input = layers.Conv2D(256, (1, 1), padding='same')(fifth_conv)
    transformer_output = network_layers.ChannelTransformer(num_heads=4, embed_dim=256, mlp_dim=512)(transformer_input)

    #decoder
    d1 = layers.UpSampling2D((2, 2), interpolation = "bilinear")(transformer_output)
    trans_up1 = layers.UpSampling2D((2, 2), interpolation = "bilinear")(transformer_output)
    trans_proj1 = layers.Conv2D(num_filters * 8, (1,1), padding='same')(trans_up1)
    fused1 = network_layers.CCA(F_g=num_filters * 8, F_x=num_filters * 8)(d1, trans_proj1)
    concat1 = layers.Concatenate()([d1, fused1])
    dec1 = network_layers.convolutional_layer(concat1, num_filters * 8, dropout = True)

    d2 = layers.UpSampling2D((2, 2), interpolation = "bilinear")(dec1)
    trans_up2 = layers.UpSampling2D((2, 2), interpolation = "bilinear")(trans_proj1)
    trans_proj2 = layers.Conv2D(num_filters * 4, (1,1), padding='same')(trans_up2)
    fused2 = network_layers.CCA(F_g=num_filters * 4, F_x=num_filters * 4)(d2, trans_proj2)
    concat2 = layers.Concatenate()([d2, fused2])
    dec2 = network_layers.convolutional_layer(concat2, num_filters * 4, dropout = True)

    d3 = layers.UpSampling2D((2, 2), interpolation = "bilinear")(dec2)
    trans_up3 = layers.UpSampling2D((2, 2), interpolation = "bilinear")(trans_proj2)
    trans_proj3 = layers.Conv2D(num_filters * 2, (1,1), padding='same')(trans_up3)
    fused3 = network_layers.CCA(F_g=num_filters * 2, F_x=num_filters * 2)(d3, trans_proj3)
    concat3 = layers.Concatenate()([d3, fused3])
    dec3 = network_layers.convolutional_layer(concat3, num_filters * 2, dropout = True)

    d4 = layers.UpSampling2D((2, 2), interpolation = "bilinear")(dec3)
    trans_up4 = layers.UpSampling2D((2, 2), interpolation = "bilinear")(trans_proj3)
    trans_proj4 = layers.Conv2D(num_filters, (1,1), padding='same')(trans_up4)
    fused4 = network_layers.CCA(F_g=num_filters, F_x=num_filters)(d4, trans_proj4)
    concat4 = layers.Concatenate()([d4, fused4])
    dec4 = network_layers.convolutional_layer(concat4, num_filters, dropout = True)

    #output
    output = layers.Conv2D(1, (1, 1), activation="sigmoid")(dec4)

    return Model(inputs, output)

In [15]:
transnet_model = UCTransNet()

In [12]:
transnet_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv2d_24 (Conv2D)          (None, 256, 256, 16)         448       ['input_2[0][0]']             
                                                                                                  
 batch_normalization_18 (Ba  (None, 256, 256, 16)         64        ['conv2d_24[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_18 (Activation)  (None, 256, 256, 16)         0         ['batch_normalization_18

## Fit on the breast cancer dataset

In [9]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import time

dice_scores = []
iou_scores = []
precision_scores = []
recall_scores = []
accuracy_scores = []
training_times = []

num_experiments = 5
input_size = (256, 256, 3)

best_dice = 0.0

for i in range(num_experiments):
    print(f"Training experiment {i+1}/{num_experiments}")

    transnet_model = UCTransNet(input_size)

    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

    transnet_model.compile(optimizer=optimizer,
                       loss=utils.tversky_loss,
                       metrics=[utils.dice_coefficient, utils.iou, "accuracy", tf.keras.metrics.Precision(name="precision_metric"), tf.keras.metrics.Recall(name="recall_metric")])

    callbacks = [
        EarlyStopping(monitor='val_dice_coefficient', patience=25, restore_best_weights=True, mode='max', verbose=0),
        ReduceLROnPlateau(monitor='val_dice_coefficient', factor=0.1, patience=7, min_lr=1e-6, mode='max', verbose=0)
    ]

    start_time = time.time()

    history = transnet_model.fit(train_dataset_breast, validation_data=val_dataset_breast, epochs=200, callbacks=callbacks, verbose=0)

    elapsed_time = time.time() - start_time
    training_times.append(elapsed_time)
    print(f"Training Time for Experiment {i+1}: {elapsed_time:.2f} seconds")

    #select best epoch in terms of dice
    best_epoch_index = np.argmax(history.history['val_dice_coefficient'])

    #get the metrics for that epoch
    final_dice = history.history['val_dice_coefficient'][best_epoch_index]
    final_iou = history.history['val_iou'][best_epoch_index]
    final_precision = history.history['val_precision_metric'][best_epoch_index]
    final_recall = history.history['val_recall_metric'][best_epoch_index]
    final_accuracy = history.history['val_accuracy'][best_epoch_index]

    #store the results
    dice_scores.append(final_dice)
    iou_scores.append(final_iou)
    precision_scores.append(final_precision)
    recall_scores.append(final_recall)
    accuracy_scores.append(final_accuracy)

    print(f"Dice Coefficient: {final_dice}", f"IoU: {final_iou}", f"Precision: {final_precision}", f"Recall: {final_recall}", f"Accuracy: {final_accuracy}")

    #save best model based on Dice score
    if final_dice > best_dice:
        best_dice = final_dice
        best_model = transnet_model
        transnet_model.save("best_uctransnet_breast.h5")
        print(f"New best model saved")

#compute mean ± std
dice_mean, dice_std = np.mean(dice_scores), np.std(dice_scores)
iou_mean, iou_std = np.mean(iou_scores), np.std(iou_scores)
precision_mean, precision_std = np.mean(precision_scores), np.std(precision_scores)
recall_mean, recall_std = np.mean(recall_scores), np.std(recall_scores)
accuracy_mean, accuracy_std = np.mean(accuracy_scores), np.std(accuracy_scores)
time_mean, time_std = np.mean(training_times), np.std(training_times)

print("Final Results (Across All Runs):")
print(f"Dice Score: {dice_mean:.2f} ± {dice_std:.2f}")
print(f"IoU Score: {iou_mean:.2f} ± {iou_std:.2f}")
print(f"Precision Score: {precision_mean:.2f} ± {precision_std:.2f}")
print(f"Recall Score: {recall_mean:.2f} ± {recall_std:.2f}")
print(f"Accuracy Score: {accuracy_mean:.2f} ± {accuracy_std:.2f}")
print(f"Training Time: {time_mean:.2f} ± {time_std:.2f} seconds")
print(f"Best Model Dice Score: {best_dice:.4f} (Saved as 'best_uctransnet_breast.h5')")

Training experiment 1/5


I0000 00:00:1739645962.580128    1592 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Training Time for Experiment 1: 1306.57 seconds
Dice Coefficient: 0.6850504875183105 IoU: 0.5326430797576904 Precision: 0.7286571264266968 Recall: 0.757993221282959 Accuracy: 0.9534263014793396


/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


New best model saved
Training experiment 2/5
Training Time for Experiment 2: 1571.21 seconds
Dice Coefficient: 0.6857962608337402 IoU: 0.5360905528068542 Precision: 0.7394036650657654 Recall: 0.7208526134490967 Accuracy: 0.9527877569198608
New best model saved
Training experiment 3/5
Training Time for Experiment 3: 938.26 seconds
Dice Coefficient: 0.6825423240661621 IoU: 0.5291728973388672 Precision: 0.7235123515129089 Recall: 0.7249425053596497 Accuracy: 0.9511235952377319
Training experiment 4/5
Training Time for Experiment 4: 1029.52 seconds
Dice Coefficient: 0.693922758102417 IoU: 0.5411513447761536 Precision: 0.7346683740615845 Recall: 0.7373513579368591 Accuracy: 0.9529690742492676
New best model saved
Training experiment 5/5
Training Time for Experiment 5: 1615.95 seconds
Dice Coefficient: 0.7044288516044617 IoU: 0.5495595335960388 Precision: 0.7478236556053162 Recall: 0.7200204730033875 Accuracy: 0.953664243221283
New best model saved
Final Results (Across All Runs):
Dice Score

### Fit on the skin cancer dataset

In [10]:
dice_scores = []
iou_scores = []
precision_scores = []
recall_scores = []
accuracy_scores = []
training_times = []

num_experiments = 5
input_size = (256, 256, 3)

best_dice = 0.0

for i in range(num_experiments):
    print(f"Training experiment {i+1}/{num_experiments}")

    transnet_model = UCTransNet(input_size)

    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

    transnet_model.compile(optimizer=optimizer,
                       loss=utils.tversky_loss,
                       metrics=[utils.dice_coefficient, utils.iou, "accuracy", tf.keras.metrics.Precision(name="precision_metric"), tf.keras.metrics.Recall(name="recall_metric")])

    callbacks = [
        EarlyStopping(monitor='val_dice_coefficient', patience=25, restore_best_weights=True, mode='max', verbose=0),
        ReduceLROnPlateau(monitor='val_dice_coefficient', factor=0.1, patience=7, min_lr=1e-6, mode='max', verbose=0)
    ]

    start_time = time.time()

    history = transnet_model.fit(train_dataset_skin, validation_data=val_dataset_skin, epochs=200, callbacks=callbacks, verbose=0)

    elapsed_time = time.time() - start_time
    training_times.append(elapsed_time)
    print(f"Training Time for Experiment {i+1}: {elapsed_time:.2f} seconds")

    #select best epoch in terms of dice
    best_epoch_index = np.argmax(history.history['val_dice_coefficient'])

    #get the metrics for that epoch
    final_dice = history.history['val_dice_coefficient'][best_epoch_index]
    final_iou = history.history['val_iou'][best_epoch_index]
    final_precision = history.history['val_precision_metric'][best_epoch_index]
    final_recall = history.history['val_recall_metric'][best_epoch_index]
    final_accuracy = history.history['val_accuracy'][best_epoch_index]

    #store the results
    dice_scores.append(final_dice)
    iou_scores.append(final_iou)
    precision_scores.append(final_precision)
    recall_scores.append(final_recall)
    accuracy_scores.append(final_accuracy)

    print(f"Dice Coefficient: {final_dice}", f"IoU: {final_iou}", f"Precision: {final_precision}", f"Recall: {final_recall}", f"Accuracy: {final_accuracy}")

    #save best model based on Dice score
    if final_dice > best_dice:
        best_dice = final_dice
        best_model = transnet_model
        transnet_model.save("best_uctransnet_skin.h5")
        print(f"New best model saved")

#compute mean ± std
dice_mean, dice_std = np.mean(dice_scores), np.std(dice_scores)
iou_mean, iou_std = np.mean(iou_scores), np.std(iou_scores)
precision_mean, precision_std = np.mean(precision_scores), np.std(precision_scores)
recall_mean, recall_std = np.mean(recall_scores), np.std(recall_scores)
accuracy_mean, accuracy_std = np.mean(accuracy_scores), np.std(accuracy_scores)
time_mean, time_std = np.mean(training_times), np.std(training_times)

print("Final Results (Across All Runs):")
print(f"Dice Score: {dice_mean:.2f} ± {dice_std:.2f}")
print(f"IoU Score: {iou_mean:.2f} ± {iou_std:.2f}")
print(f"Precision Score: {precision_mean:.2f} ± {precision_std:.2f}")
print(f"Recall Score: {recall_mean:.2f} ± {recall_std:.2f}")
print(f"Accuracy Score: {accuracy_mean:.2f} ± {accuracy_std:.2f}")
print(f"Training Time: {time_mean:.2f} ± {time_std:.2f} seconds")
print(f"Best Model Dice Score: {best_dice:.4f} (Saved as 'best_uctransnet_skin.h5')")

Training experiment 1/5


I0000 00:00:1739475065.193683   12951 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Training Time for Experiment 1: 4170.69 seconds
Dice Coefficient: 0.7984616756439209 IoU: 0.6737697720527649 Precision: 0.8398386240005493 Recall: 0.7707529067993164 Accuracy: 0.9345341920852661


/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


New best model saved
Training experiment 2/5
Training Time for Experiment 2: 5120.38 seconds
Dice Coefficient: 0.8043211102485657 IoU: 0.6823873519897461 Precision: 0.8263492584228516 Recall: 0.7850642204284668 Accuracy: 0.9338929057121277
New best model saved
Training experiment 3/5
Training Time for Experiment 3: 6173.03 seconds
Dice Coefficient: 0.8124055862426758 IoU: 0.6945287585258484 Precision: 0.8760154247283936 Recall: 0.7860246300697327 Accuracy: 0.943415105342865
New best model saved
Training experiment 4/5
Training Time for Experiment 4: 4531.30 seconds
Dice Coefficient: 0.8075373768806458 IoU: 0.6883435249328613 Precision: 0.8253182172775269 Recall: 0.8222740292549133 Accuracy: 0.9387778639793396
Training experiment 5/5
Training Time for Experiment 5: 4951.73 seconds
Dice Coefficient: 0.7912952303886414 IoU: 0.6643930673599243 Precision: 0.8210409879684448 Recall: 0.7880485653877258 Accuracy: 0.9332371950149536
Final Results (Across All Runs):
Dice Score: 0.80 ± 0.01
IoU S

## Fit on the brain cancer dataset

In [9]:
dice_scores = []
iou_scores = []
precision_scores = []
recall_scores = []
accuracy_scores = []
training_times = []

num_experiments = 5
input_size = (256, 256, 3)

best_dice = 0.0

for i in range(num_experiments):
    print(f"Training experiment {i+1}/{num_experiments}")

    transnet_model = UCTransNet(input_size)

    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

    transnet_model.compile(optimizer=optimizer,
                       loss=utils.tversky_loss,
                       metrics=[utils.dice_coefficient, utils.iou, "accuracy", tf.keras.metrics.Precision(name="precision_metric"), tf.keras.metrics.Recall(name="recall_metric")])

    callbacks = [
        EarlyStopping(monitor='val_dice_coefficient', patience=25, restore_best_weights=True, mode='max', verbose=0),
        ReduceLROnPlateau(monitor='val_dice_coefficient', factor=0.1, patience=7, min_lr=1e-6, mode='max', verbose=0)
    ]

    start_time = time.time()

    history = transnet_model.fit(train_dataset_brain, validation_data=val_dataset_brain, epochs=200, callbacks=callbacks, verbose=0)

    elapsed_time = time.time() - start_time
    training_times.append(elapsed_time)
    print(f"Training Time for Experiment {i+1}: {elapsed_time:.2f} seconds")

    #select best epoch in terms of dice
    best_epoch_index = np.argmax(history.history['val_dice_coefficient'])

    #get the metrics for that epoch
    final_dice = history.history['val_dice_coefficient'][best_epoch_index]
    final_iou = history.history['val_iou'][best_epoch_index]
    final_precision = history.history['val_precision_metric'][best_epoch_index]
    final_recall = history.history['val_recall_metric'][best_epoch_index]
    final_accuracy = history.history['val_accuracy'][best_epoch_index]

    #store the results
    dice_scores.append(final_dice)
    iou_scores.append(final_iou)
    precision_scores.append(final_precision)
    recall_scores.append(final_recall)
    accuracy_scores.append(final_accuracy)

    print(f"Dice Coefficient: {final_dice}", f"IoU: {final_iou}", f"Precision: {final_precision}", f"Recall: {final_recall}", f"Accuracy: {final_accuracy}")

    #save best model based on Dice score
    if final_dice > best_dice:
        best_dice = final_dice
        best_model = transnet_model
        transnet_model.save("best_uctransnet_brain.h5")
        print(f"New best model saved")

#compute mean ± std
dice_mean, dice_std = np.mean(dice_scores), np.std(dice_scores)
iou_mean, iou_std = np.mean(iou_scores), np.std(iou_scores)
precision_mean, precision_std = np.mean(precision_scores), np.std(precision_scores)
recall_mean, recall_std = np.mean(recall_scores), np.std(recall_scores)
accuracy_mean, accuracy_std = np.mean(accuracy_scores), np.std(accuracy_scores)
time_mean, time_std = np.mean(training_times), np.std(training_times)

print("Final Results (Across All Runs):")
print(f"Dice Score: {dice_mean:.2f} ± {dice_std:.2f}")
print(f"IoU Score: {iou_mean:.2f} ± {iou_std:.2f}")
print(f"Precision Score: {precision_mean:.2f} ± {precision_std:.2f}")
print(f"Recall Score: {recall_mean:.2f} ± {recall_std:.2f}")
print(f"Accuracy Score: {accuracy_mean:.2f} ± {accuracy_std:.2f}")
print(f"Training Time: {time_mean:.2f} ± {time_std:.2f} seconds")
print(f"Best Model Dice Score: {best_dice:.4f} (Saved as 'best_uctransnet_brain.h5')")

Training experiment 1/5


I0000 00:00:1739532427.837396   10269 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Training Time for Experiment 1: 5228.23 seconds
Dice Coefficient: 0.6648396849632263 IoU: 0.5081547498703003 Precision: 0.7275176048278809 Recall: 0.6342777013778687 Accuracy: 0.9887841939926147


/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


New best model saved
Training experiment 2/5
Training Time for Experiment 2: 4181.70 seconds
Dice Coefficient: 0.6447569727897644 IoU: 0.4875197410583496 Precision: 0.7137389779090881 Recall: 0.6043141484260559 Accuracy: 0.9881910681724548
Training experiment 3/5
Training Time for Experiment 3: 3177.79 seconds
Dice Coefficient: 0.6494719982147217 IoU: 0.4903183579444885 Precision: 0.6831292510032654 Recall: 0.6385229229927063 Accuracy: 0.9877679944038391
Training experiment 4/5
Training Time for Experiment 4: 4966.76 seconds
Dice Coefficient: 0.6655905842781067 IoU: 0.5098310112953186 Precision: 0.7059208154678345 Recall: 0.6463741660118103 Accuracy: 0.9884274005889893
New best model saved
Training experiment 5/5
Training Time for Experiment 5: 4823.57 seconds
Dice Coefficient: 0.6615387797355652 IoU: 0.5049335956573486 Precision: 0.7428768277168274 Recall: 0.6095962524414062 Accuracy: 0.9888470768928528
Final Results (Across All Runs):
Dice Score: 0.66 ± 0.01
IoU Score: 0.50 ± 0.01
Pr